In [1]:
!pip install -q duckdb --upgrade
import sys
sys.exit(0)

sys.exit called with value 0. The interpreter will be restarted.


In [2]:
Nbr_Files_to_Download = 2623
web                   = "/lakehouse/default/Files/Daily_Reports/"
source                = "/lakehouse/default/Files/csv/"
output_path           = "/lakehouse/default/Files/excel/NEM-Registration-and-Exemption-List.xls"
metadata              = "/lakehouse/default/Files/meta.db"
tables                = "/lakehouse/default/Tables"
schema                = 'aemo'
catalog               = 'db'

In [3]:
import duckdb
from   duckdb import ParserException
import os
import json
import uuid
import time

In [4]:
con = duckdb.connect()
con.sql(f""" force install ducklake from core_nightly ;INSTALL spatial; LOAD spatial;  """)
con.sql(f""" ATTACH or replace 'ducklake:{metadata}' AS {catalog} (DATA_PATH '{tables}') ;USE db """)
con.sql("""  create schema if not exists aemo ; use aemo ; """)
con.sql("""  
                CALL set_option('parquet_row_group_size', '2_000_000' );
                CALL set_option('parquet_compression', 'zstd' );
         """)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [5]:
def table_exists(table_name: str, schema: str = schema) -> bool:
    result = con.execute(f"""
        SELECT COUNT(*) > 0 AS exists
        FROM information_schema.tables
        WHERE table_schema = '{schema}' AND table_name = '{table_name}'
    """).fetchone()[0]
    return result

# ***Download  Open Data***

In [6]:
# @title
import re ,shutil,glob
from   urllib.request import urlopen
import requests
import multiprocessing
import os
from   shutil import unpack_archive
def download(url,Path,total_files):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique, reverse=True)
    current =  [os.path.basename(x) for x in glob.glob(Path+'*.zip')]
    files_to_upload = sorted(list(set(filelist) - set(current)),reverse=True)
    files_to_upload = list(dict.fromkeys(files_to_upload))[:total_files]
    print(str(len(files_to_upload)) + ' New File Loaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

In [7]:
# @title
def uncompress(x):
        unpack_archive(str(web+x), str(source), 'zip')
def unzip(Source, Destination,Nbr_Files_to_Download):
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True)
    filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    ### checl the unzipped files already
    current = [os.path.basename(x) for x in glob.glob(Destination+'*.CSV')]
    current = [w.replace('.CSV','.zip') for w in current]
    #unzip only the delta
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))
    print(str(len(files_to_upload)) + ' New File uncompressed')
    if len(files_to_upload) != 0 :
      with multiprocessing.Pool() as pool:
       for _ in pool.imap_unordered(uncompress, files_to_upload, chunksize=1):
         pass
      return "done"
    else:
     return "nothing to see here"

In [8]:
%%time
download("https://nemweb.com.au/Reports/Current/Daily_Reports/", web, Nbr_Files_to_Download)
unzip(web, source,Nbr_Files_to_Download)

1 New File Loaded
1 New File uncompressed
CPU times: user 97.1 ms, sys: 25.9 ms, total: 123 ms
Wall time: 8.13 s


'done'

In [9]:
url = "https://www.aemo.com.au/-/media/Files/Electricity/NEM/Participant_Information/NEM-Registration-and-Exemption-List.xls"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
print("Downloading file...")
response = requests.get(url, headers=headers, stream=True)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print(f"File saved to: {output_path}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File saved to: /lakehouse/default/Files/excel/NEM-Registration-and-Exemption-List.xls


# SQL

In [10]:
scada_sql = f"""
WITH raw AS (
    FROM read_csv(
        getvariable('list_of_files_scada'),
        skip = 1,
        header = 0,
        all_varchar = 1,
        columns = {{
            'I': 'VARCHAR', 'UNIT': 'VARCHAR', 'XX': 'VARCHAR', 'VERSION': 'VARCHAR',
            'SETTLEMENTDATE': 'VARCHAR', 'RUNNO': 'VARCHAR', 'DUID': 'VARCHAR', 'INTERVENTION': 'VARCHAR',
            'DISPATCHMODE': 'VARCHAR', 'AGCSTATUS': 'VARCHAR', 'INITIALMW': 'VARCHAR', 'TOTALCLEARED': 'VARCHAR',
            'RAMPDOWNRATE': 'VARCHAR', 'RAMPUPRATE': 'VARCHAR', 'LOWER5MIN': 'VARCHAR', 'LOWER60SEC': 'VARCHAR',
            'LOWER6SEC': 'VARCHAR', 'RAISE5MIN': 'VARCHAR', 'RAISE60SEC': 'VARCHAR', 'RAISE6SEC': 'VARCHAR',
            'MARGINAL5MINVALUE': 'VARCHAR', 'MARGINAL60SECVALUE': 'VARCHAR', 'MARGINAL6SECVALUE': 'VARCHAR', 'MARGINALVALUE': 'VARCHAR',
            'VIOLATION5MINDEGREE': 'VARCHAR', 'VIOLATION60SECDEGREE': 'VARCHAR', 'VIOLATION6SECDEGREE': 'VARCHAR', 'VIOLATIONDEGREE': 'VARCHAR',
            'LOWERREG': 'VARCHAR', 'RAISEREG': 'VARCHAR', 'AVAILABILITY': 'VARCHAR', 'RAISE6SECFLAGS': 'VARCHAR',
            'RAISE60SECFLAGS': 'VARCHAR', 'RAISE5MINFLAGS': 'VARCHAR', 'RAISEREGFLAGS': 'VARCHAR', 'LOWER6SECFLAGS': 'VARCHAR',
            'LOWER60SECFLAGS': 'VARCHAR', 'LOWER5MINFLAGS': 'VARCHAR', 'LOWERREGFLAGS': 'VARCHAR', 'RAISEREGAVAILABILITY': 'VARCHAR',
            'RAISEREGENABLEMENTMAX': 'VARCHAR', 'RAISEREGENABLEMENTMIN': 'VARCHAR', 'LOWERREGAVAILABILITY': 'VARCHAR', 'LOWERREGENABLEMENTMAX': 'VARCHAR',
            'LOWERREGENABLEMENTMIN': 'VARCHAR', 'RAISE6SECACTUALAVAILABILITY': 'VARCHAR', 'RAISE60SECACTUALAVAILABILITY': 'VARCHAR', 'RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
            'RAISEREGACTUALAVAILABILITY': 'VARCHAR', 'LOWER6SECACTUALAVAILABILITY': 'VARCHAR', 'LOWER60SECACTUALAVAILABILITY': 'VARCHAR', 'LOWER5MINACTUALAVAILABILITY': 'VARCHAR',
            'LOWERREGACTUALAVAILABILITY': 'VARCHAR'
        }},
        filename = 1,
        null_padding = true,
        ignore_errors = 1,
        auto_detect = false
    )
    WHERE
        I = 'D'
        AND UNIT = 'DUNIT'
        AND VERSION = 3
)
SELECT
    UNIT,
    DUID,
    CAST(columns(* EXCLUDE (DUID, UNIT, SETTLEMENTDATE, I, XX, filename)) AS DOUBLE),
    parse_filename(filename) AS file,
    CAST(SETTLEMENTDATE AS TIMESTAMP) AS SETTLEMENTDATE,
    CAST(SETTLEMENTDATE AS DATE) AS DATE,
    CAST(YEAR(CAST(SETTLEMENTDATE AS TIMESTAMP)) AS INT16) AS YEAR
FROM raw
"""

In [11]:
price_sql =f"""
WITH raw AS (
    FROM read_csv(
        getvariable('list_of_files_price'),
        skip = 1,
        header = 0,
        all_varchar = 1,
  columns={{
  'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
  'REGIONID': 'VARCHAR','INTERVENTION': 'VARCHAR','RRP': 'VARCHAR','EEP': 'VARCHAR','ROP': 'VARCHAR','APCFLAG': 'VARCHAR',
  'MARKETSUSPENDEDFLAG': 'VARCHAR','TOTALDEMAND': 'VARCHAR','DEMANDFORECAST': 'VARCHAR','DISPATCHABLEGENERATION': 'VARCHAR',
  'DISPATCHABLELOAD': 'VARCHAR','NETINTERCHANGE': 'VARCHAR','EXCESSGENERATION': 'VARCHAR','LOWER5MINDISPATCH': 'VARCHAR',
  'LOWER5MINIMPORT': 'VARCHAR','LOWER5MINLOCALDISPATCH': 'VARCHAR','LOWER5MINLOCALPRICE': 'VARCHAR','LOWER5MINLOCALREQ': 'VARCHAR',
  'LOWER5MINPRICE': 'VARCHAR','LOWER5MINREQ': 'VARCHAR','LOWER5MINSUPPLYPRICE': 'VARCHAR','LOWER60SECDISPATCH': 'VARCHAR','LOWER60SECIMPORT': 'VARCHAR',
  'LOWER60SECLOCALDISPATCH': 'VARCHAR','LOWER60SECLOCALPRICE': 'VARCHAR','LOWER60SECLOCALREQ': 'VARCHAR','LOWER60SECPRICE': 'VARCHAR',
  'LOWER60SECREQ': 'VARCHAR','LOWER60SECSUPPLYPRICE': 'VARCHAR','LOWER6SECDISPATCH': 'VARCHAR','LOWER6SECIMPORT': 'VARCHAR',
  'LOWER6SECLOCALDISPATCH': 'VARCHAR','LOWER6SECLOCALPRICE': 'VARCHAR','LOWER6SECLOCALREQ': 'VARCHAR','LOWER6SECPRICE': 'VARCHAR',
  'LOWER6SECREQ': 'VARCHAR','LOWER6SECSUPPLYPRICE': 'VARCHAR','RAISE5MINDISPATCH': 'VARCHAR','RAISE5MINIMPORT': 'VARCHAR',
  'RAISE5MINLOCALDISPATCH': 'VARCHAR','RAISE5MINLOCALPRICE': 'VARCHAR','RAISE5MINLOCALREQ': 'VARCHAR','RAISE5MINPRICE': 'VARCHAR',
  'RAISE5MINREQ': 'VARCHAR','RAISE5MINSUPPLYPRICE': 'VARCHAR','RAISE60SECDISPATCH': 'VARCHAR','RAISE60SECIMPORT': 'VARCHAR',
  'RAISE60SECLOCALDISPATCH': 'VARCHAR','RAISE60SECLOCALPRICE': 'VARCHAR','RAISE60SECLOCALREQ': 'VARCHAR','RAISE60SECPRICE': 'VARCHAR',
  'RAISE60SECREQ': 'VARCHAR','RAISE60SECSUPPLYPRICE': 'VARCHAR','RAISE6SECDISPATCH': 'VARCHAR','RAISE6SECIMPORT': 'VARCHAR',
  'RAISE6SECLOCALDISPATCH': 'VARCHAR','RAISE6SECLOCALPRICE': 'VARCHAR','RAISE6SECLOCALREQ': 'VARCHAR','RAISE6SECPRICE': 'VARCHAR',
  'RAISE6SECREQ': 'VARCHAR','RAISE6SECSUPPLYPRICE': 'VARCHAR','AGGREGATEDISPATCHERROR': 'VARCHAR','AVAILABLEGENERATION': 'VARCHAR',
  'AVAILABLELOAD': 'VARCHAR','INITIALSUPPLY': 'VARCHAR','CLEAREDSUPPLY': 'VARCHAR','LOWERREGIMPORT': 'VARCHAR','LOWERREGLOCALDISPATCH': 'VARCHAR',
  'LOWERREGLOCALREQ': 'VARCHAR','LOWERREGREQ': 'VARCHAR','RAISEREGIMPORT': 'VARCHAR','RAISEREGLOCALDISPATCH': 'VARCHAR','RAISEREGLOCALREQ': 'VARCHAR',
  'RAISEREGREQ': 'VARCHAR','RAISE5MINLOCALVIOLATION': 'VARCHAR','RAISEREGLOCALVIOLATION': 'VARCHAR','RAISE60SECLOCALVIOLATION': 'VARCHAR',
  'RAISE6SECLOCALVIOLATION': 'VARCHAR','LOWER5MINLOCALVIOLATION': 'VARCHAR','LOWERREGLOCALVIOLATION': 'VARCHAR','LOWER60SECLOCALVIOLATION': 'VARCHAR',
  'LOWER6SECLOCALVIOLATION': 'VARCHAR','RAISE5MINVIOLATION': 'VARCHAR','RAISEREGVIOLATION': 'VARCHAR','RAISE60SECVIOLATION': 'VARCHAR',
  'RAISE6SECVIOLATION': 'VARCHAR','LOWER5MINVIOLATION': 'VARCHAR','LOWERREGVIOLATION': 'VARCHAR','LOWER60SECVIOLATION': 'VARCHAR',
  'LOWER6SECVIOLATION': 'VARCHAR','RAISE6SECRRP': 'VARCHAR','RAISE6SECROP': 'VARCHAR','RAISE6SECAPCFLAG': 'VARCHAR','RAISE60SECRRP': 'VARCHAR',
  'RAISE60SECROP': 'VARCHAR','RAISE60SECAPCFLAG': 'VARCHAR','RAISE5MINRRP': 'VARCHAR','RAISE5MINROP': 'VARCHAR','RAISE5MINAPCFLAG': 'VARCHAR',
  'RAISEREGRRP': 'VARCHAR','RAISEREGROP': 'VARCHAR','RAISEREGAPCFLAG': 'VARCHAR','LOWER6SECRRP': 'VARCHAR','LOWER6SECROP': 'VARCHAR',
  'LOWER6SECAPCFLAG': 'VARCHAR','LOWER60SECRRP': 'VARCHAR','LOWER60SECROP': 'VARCHAR','LOWER60SECAPCFLAG': 'VARCHAR','LOWER5MINRRP': 'VARCHAR',
  'LOWER5MINROP': 'VARCHAR','LOWER5MINAPCFLAG': 'VARCHAR','LOWERREGRRP': 'VARCHAR','LOWERREGROP': 'VARCHAR','LOWERREGAPCFLAG': 'VARCHAR',
  'RAISE6SECACTUALAVAILABILITY': 'VARCHAR','RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
  'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR','LOWER60SECACTUALAVAILABILITY': 'VARCHAR',
  'LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR','LORSURPLUS': 'VARCHAR','LRCSURPLUS': 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and UNIT ='DREGION' AND VERSION = 3
                  )
    select
    UNIT,
    REGIONID,
    cast(columns(*exclude(REGIONID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    cast (SETTLEMENTDATE as TIMESTAMP) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as DATE,
	  cast(year (cast (SETTLEMENTDATE as timestamp)) AS INT16) as YEAR
    from raw
      """

In [12]:
calendar_sql =f"""
            SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2026-12-31' as date), interval 1 day)) as date) as date,
            EXTRACT(year from date) as year,
            EXTRACT(month from date) as month
            """

In [13]:
mstdatetime_sql =f"""
SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2024-12-31' as date), interval 5 minute)) as TIMESTAMPTZ) as SETTLEMENTDATE,
           strftime(SETTLEMENTDATE, '%I:%M:%S %p') as time,
           cast(SETTLEMENTDATE as date ) as date,
           EXTRACT(year from date) as year,
           EXTRACT(month from date) as month
           """

In [14]:
duid_sql=f"""
WITH NEM_XLS AS (
  SELECT * FROM st_read(
    '{output_path}',
    layer = 'PU and Scheduled Loads',
    open_options = ['HEADERS=FORCE']
  )
),
NEM_DATA AS (
  SELECT
    Region,
    DUID,
    FIRST("Fuel Source - Descriptor") AS FuelSourceDescriptor,
    FIRST(Participant) AS Participant
  FROM NEM_XLS
  GROUP BY ALL
),

WA_FACILITIES AS (
  SELECT
    'WA1' AS Region,
    "Facility Code" AS DUID,
    "Participant Name" AS Participant
  FROM read_csv_auto(
    'https://data.wa.aemo.com.au/datafiles/post-facilities/facilities.csv'
  )
),

-- Join WA facilities with local technology mapping (from GitHub)
WA_ENERGY_MAP AS (
  SELECT *
  FROM read_csv_auto(
    'https://github.com/djouallah/aemo_fabric/raw/main/WA_ENERGY.csv',
    header = TRUE
  )
),

-- Combine WA facility data with mapped tech type
WA_DATA AS (
  SELECT
    w.Region,
    w.DUID,
    COALESCE(e.TECHNOLOGY, 'Unknown') AS FuelSourceDescriptor,
    w.Participant
  FROM WA_FACILITIES w
  LEFT JOIN WA_ENERGY_MAP e ON w.DUID = e.DUID
),

-- Define region to state mapping
REGION_STATES AS (
  SELECT * FROM (
    VALUES
      ('WA1', 'Western Australia'),
      ('QLD1', 'Queensland'),
      ('NSW1', 'New South Wales'),
      ('TAS1', 'Tasmania'),
      ('SA1', 'South Australia'),
      ('VIC1', 'Victoria')
  ) AS t(RegionID, States)
),

-- Union all DUID sources and map to states
UNIONED_DATA AS (
  SELECT * FROM NEM_DATA
  UNION BY NAME
  SELECT * FROM WA_DATA
),

-- Final joined result with states
FINAL_RESULT AS (
  SELECT
    rs.States,
    TRIM(u.DUID) AS DUID,
    MIN(u.Region) AS Region,
    MIN(u.FuelSourceDescriptor) AS FuelSourceDescriptor,
    MIN(u.Participant) AS Participant
  FROM UNIONED_DATA u
  JOIN REGION_STATES rs ON u.Region = rs.RegionID
  GROUP BY ALL
)

-- Output final table
SELECT * FROM FINAL_RESULT;
"""

In [15]:
summary_sql = """
        select
        s.DATE as date,
        cast(strftime(s.SETTLEMENTDATE, '%H%M') AS INT16)                       as time ,
        (select max(cast(settlementdate as TIMESTAMPTZ) ) from scada)           as cutoff ,
        s.DUID,
        max(s.INITIALMW)                                                        as mw,
        max(p.RRP)                                                              as price
      from  scada   s
            LEFT JOIN duid d    ON s.DUID = d.DUID
            LEFT JOIN price   p ON s.SETTLEMENTDATE = p.SETTLEMENTDATE AND d.Region = p.REGIONID
      where
        s.INTERVENTION = 0
        and INITIALMW <> 0
        and p.INTERVENTION = 0
        --and s.YEAR = 2020
      group by all
      """

# ***Run SQL Tasks***

In [16]:
def map_type_ducklake(ducklake_type):
    """Map DuckLake column types to Delta Lake types"""
    if 'int' in ducklake_type.lower():
        if '16' in ducklake_type:
            return "short"
        elif '32' in ducklake_type:
            return "integer"
        else:
            return "long"
    elif 'float' in ducklake_type.lower() or 'double' in ducklake_type.lower():
        return "double"
    elif 'string' in ducklake_type.lower() or 'varchar' in ducklake_type.lower():
        return "string"
    elif 'boolean' in ducklake_type.lower():
        return "boolean"
    elif 'date' in ducklake_type.lower():
        return "date"
    elif 'timestamp' in ducklake_type.lower():
        return "timestamp"
    else:
        raise NotImplementedError(f"Unsupported type: {ducklake_type}")


def get_current_schema(con, table_id):
    """Get schema from ducklake_column for given table_id"""
    columns = con.execute(f"""
        SELECT column_name, column_type 
        FROM __ducklake_metadata_db.ducklake_column 
        WHERE table_id = {table_id} AND end_snapshot IS NULL
        ORDER BY column_order
    """).fetchall()

    return [{
        "name": name,
        "type": map_type_ducklake(col_type),
        "nullable": True,
        "metadata": {}
    } for name, col_type in columns]


def get_data_files_with_stats(con, table_id, table_root):
    """Get active data files + their stats from ducklake_data_file and ducklake_file_column_statistics"""
    files = con.execute(f"""
        SELECT df.data_file_id, df.path, df.file_size_bytes 
        FROM __ducklake_metadata_db.ducklake_data_file df
        WHERE df.table_id = {table_id} AND df.end_snapshot IS NULL
    """).fetchall()

    data_files = []
    for data_file_id, path, size in files:
        full_path = os.path.join(table_root, path)

        # Get stats per column
        stats_rows = con.execute(f"""
            SELECT c.column_name, s.min_value, s.max_value, s.null_count, s.value_count
            FROM __ducklake_metadata_db.ducklake_file_column_statistics s
            JOIN __ducklake_metadata_db.ducklake_column c ON s.column_id = c.column_id AND c.end_snapshot IS NULL
            WHERE s.data_file_id = {data_file_id} AND s.table_id = {table_id}
        """).fetchall()

        min_values = {}
        max_values = {}
        null_count = {}
        num_records = None

        for col_name, min_val, max_val, nulls, vals in stats_rows:
            min_values[col_name] = str(min_val) if min_val is not None else None
            max_values[col_name] = str(max_val) if max_val is not None else None
            null_count[col_name] = int(nulls) if nulls is not None else 0
            num_records = int(vals) if vals is not None else 0

        stats = {
            "numRecords": num_records,
            "minValues": min_values,
            "maxValues": max_values,
            "nullCount": null_count
        }

        data_files.append({
            "path": full_path,
            "size": size,
            "stats": stats
        })

    return data_files


def get_latest_snapshot_info(con, table_id):
    """Get latest snapshot time for the table"""
    result = con.execute(f"""
        SELECT MAX(snapshot_time) 
        FROM __ducklake_metadata_db.ducklake_snapshot 
        WHERE snapshot_id IN (
            SELECT begin_snapshot FROM __ducklake_metadata_db.ducklake_table 
            WHERE table_id = {table_id}
        )
    """).fetchone()

    snapshot_time = result[0] if result and result[0] else None
    return snapshot_time

def has_delete_files(con, table_id):
    """
    Returns True if the table has any delete files in the latest snapshot.
    Uses DuckLake metadata only.
    """
    result = con.execute(f"""
        SELECT COUNT(*) 
        FROM __ducklake_metadata_db.ducklake_delete_file 
        WHERE table_id = {table_id}
    """).fetchone()
    
    return result[0] > 0
def generate_latest_delta_log(schema, table_name):

    # Step 1: Get table_id from ducklake_table
    result = con.execute(f"""
        SELECT table_id FROM __ducklake_metadata_db.ducklake_table 
        WHERE table_name = '{table_name}' AND schema_id IN (
            SELECT schema_id FROM __ducklake_metadata_db.ducklake_schema WHERE schema_name = '{schema}'
        ) AND end_snapshot IS NULL
    """).fetchone()

    if not result:
        raise ValueError(f"Table '{schema}.{table_name}' not found in DuckLake metadata")

    table_id = result[0]

    # Assume you already have a DuckDB connection `con` and a `table_id`
    if has_delete_files(con, table_id):
        print("⚠️ Delete files found — need to materialize table")
        con.sql(f""" create or replace table {table_name} as select * from {table_name} """)
    else:
        print("✅ No delete files — safe to use existing data")

    # Step 2: Get table root path
    table_root = con.sql(f" select parse_dirpath(data_file) from ducklake_list_files('{catalog}', '{table_name}') limit 1 ").fetchone()[0]

    # Step 3: Get schema
    schema_fields = get_current_schema(con, table_id)

    # Step 4: Get data files with stats
    data_files = get_data_files_with_stats(con, table_id, table_root)

    # Step 5: Get snapshot info
    snapshot_time = get_latest_snapshot_info(con, table_id)
    snapshot_time_str = snapshot_time.isoformat() if snapshot_time else "unknown"

    # Step 6: Prepare Delta log
    delta_log = os.path.join(table_root, "_delta_log")
    version = 0  # Overwrite mode
    log_file = os.path.join(delta_log, f"{version:020d}.json")

    # Clean out old logs
    if os.path.exists(delta_log):
        for f in os.listdir(delta_log):
            os.remove(os.path.join(delta_log, f))
    else:
        os.makedirs(delta_log, exist_ok=True)

    # Step 7: Build Delta log content
    protocol = {"protocol": {"minReaderVersion": 1, "minWriterVersion": 2}}

    metadata = {
        "metaData": {
            "id": str(uuid.uuid4()),
            "format": {"provider": "parquet", "options": {}},
            "schemaString": json.dumps({
                "type": "struct",
                "fields": schema_fields
            }),
            "partitionColumns": [],
            "createdTime": int(time.time() * 1000),
            "configuration": {
                "writer": "ducklake",
                "table_ud": str(table_id),
                "name": table_name,
                "schema": schema,
                "snapshot_time": snapshot_time_str
            }
        }
    }

    adds = []
    for f in data_files:
        adds.append({
            "add": {
                "path": os.path.relpath(f["path"], table_root).replace("\\", "/"),
                "partitionValues": {},
                "size": f["size"],
                "modificationTime": int(os.path.getmtime(f["path"]) * 1000) if os.path.exists(f["path"]) else int(time.time() * 1000),
                "dataChange": True,
                "stats": json.dumps(f["stats"])  # ⬅️ Serialized as string
            }
        })

    # Step 8: Write the Delta log
    with open(log_file, "w") as f:
        f.write(json.dumps(protocol) + "\n")
        f.write(json.dumps(metadata) + "\n")
        for a in adds:
            f.write(json.dumps(a) + "\n")

    print("Wrote Delta snapshot with column stats and custom tags")

In [17]:
def run_sql_tasks(task_list):
    for table, mode in task_list:
        sql_variable_name = f"{table}_sql"
        full_table_name = f"{table}"
        con.sql("checkpoint __ducklake_metadata_db")
        try:
            if sql_variable_name not in globals():
                raise ValueError(f"SQL variable '{sql_variable_name}' is not defined.")
            if mode == "ignore":
                if table_exists(full_table_name):
                    print(f"Table '{full_table_name}' already exists. Skipping.")
                    continue
                con.execute(f'create or replace table {table} as' + globals()[sql_variable_name])
                generate_latest_delta_log(schema,table)
                print(f"Table '{full_table_name}' updated")
            elif mode == "overwrite":
                con.execute(f'create or replace table {table} as' + globals()[sql_variable_name])
                generate_latest_delta_log(schema,table)
                print(f"Table '{full_table_name}' overwritten")
            elif mode == "append":
                if not table_exists(full_table_name):
                    con.sql(f"""SET VARIABLE list_of_files_{table} =
                        ( SELECT list(file) FROM (SELECT file FROM glob('{source}/*.CSV')limit {Nbr_Files_to_Download} ))  """)
                    con.sql(f'create or replace table {table} as' + globals()[sql_variable_name])
                    generate_latest_delta_log(schema,table)
                    print(f"Table '{full_table_name}' updated ")
                else:
                    con.sql(f""" SET VARIABLE list_of_files_{table} = (
                                            WITH xxxx AS (
                                                SELECT file FROM glob('{source}*.CSV')
                                                WHERE parse_filename(file) NOT IN (
                                                    SELECT DISTINCT file FROM {table} 
                                                )    limit {Nbr_Files_to_Download}
                                            )
                                            SELECT list(file) FROM xxxx 
                                        );""")
                    con.execute(f'insert into {table} ' + globals()[sql_variable_name])
                    generate_latest_delta_log(schema,table)
                    print(f"Table '{full_table_name}' updated")
            else:
                raise ValueError(f"Unsupported mode: '{mode}'. Use 'append', 'overwrite', or 'ignore'.")
        except ParserException as pe:
            print(f"[Parser Error] Failed to parse SQL for table '{table}': {pe}")
            return
        except duckdb.Error as e:
            print(f"[DuckDB Error] Failed to execute SQL for table '{table}': {e}")
            return
        except Exception as e:
            print(f"[Unexpected Error] During processing table '{table}': {e}")
            return

In [18]:
%%time
run_sql_tasks(
    [('calendar','ignore'),
    ('mstdatetime','ignore'),
    ('scada','append'),
    ('price','append'),
    ('duid','overwrite'),
    ('summary','overwrite')]
    )

Table 'calendar' already exists. Skipping.
Table 'mstdatetime' already exists. Skipping.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ No delete files — safe to use existing data
Wrote Delta snapshot with column stats and custom tags
Table 'scada' updated


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ No delete files — safe to use existing data
Wrote Delta snapshot with column stats and custom tags
Table 'price' updated
✅ No delete files — safe to use existing data
Wrote Delta snapshot with column stats and custom tags
Table 'duid' overwritten


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

No such comm: c1ffbae17efa4313ad0198d42fa52b58
No such comm: 78339da3344646a880e5a95fa42d9aff
No such comm: c45c8957e83147ff85a8c176c367049d
No such comm: c2709212d9dd4b5988b878a5ba43eae6
No such comm: 8d9ba10c323941eb9d3342a2e15d8fdf
No such comm: 442ebddbad24414c8663bbecdf3784c3


✅ No delete files — safe to use existing data
Wrote Delta snapshot with column stats and custom tags
Table 'summary' overwritten
CPU times: user 3min 40s, sys: 8.48 s, total: 3min 48s
Wall time: 2min 24s


In [19]:
con.sql(" select count(*) from scada ")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    304939388 │
└──────────────┘

In [20]:
con.sql(" select count(*) from price ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      4000590 │
└──────────────┘

In [21]:
con.sql(" select count(*) from summary ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    121311987 │
└──────────────┘

In [22]:
duckdb.sql(" select count(*) from  delta_scan('/lakehouse/default/Tables/aemo/summary') ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    121311987 │
└──────────────┘

In [23]:
con.close()